# Imports

In [1]:
#major_imports
from glob import glob 
from klampt import IKObjective
from klampt import WorldModel,Geometry3D
from klampt import vis
from klampt.math import so3,se3
from klampt.io import resource
from klampt.math import vectorops,so3
from klampt.model.collide import WorldCollider
from klampt.model import ik
from klampt.plan.cspace import CSpace,MotionPlan
from klampt.model.trajectory import RobotTrajectory,Trajectory
from klampt.math.vectorops import interpolate
from klampt.plan.cspace import CSpace,MotionPlan
from planning.disinfection3d import DisinfectionProblem
from planning.robot_cspaces import Robot3DCSpace,CSpaceObstacleSolver,UnreachablePointsError
import pickle
import os
import numpy as np
from planning.tsp_solver_wrapper import runTSP
import networkx as nx
import klampt
from klampt.plan.robotcspace import RobotCSpace
from klampt.plan import cspace
from klampt.model import collide
import klampt
from klampt.plan import cspace,robotplanning
from klampt.io import resource
import time
import trimesh as tm
from alphashape import alphashape
import shapely
from planning.auxiliary_functions import setup_robot_and_light,get_bounds,load_relevant_info
from planning.auxiliary_functions import extract_milestone_tour,collisionChecker,find_n_ik_solutions
from planning.auxiliary_functions import iterative_farthest_point,euclidean_trelis_solution,set_robot_link_collision_margins
from planning.auxiliary_functions import interp,compute_actual_distance,heuristic_penalized_trelis_solution
from planning.auxiliary_functions import heuristic_ee_trelis_solution,calculate_path_length_from_configuration_sequence
from tqdm import tqdm
from random import choice
from klampt.model.trajectory import RobotTrajectory,Trajectory


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
***  klampt.vis: using Qt5 as the visualization backend  ***


In [ ]:
roadmaps = glob('./Distance/*/*/*/armbot_roadmap_330_divs.p')

total_dofs  = 11
robot_name = 'armbot'
world,robot,lamp,roadmap,sampling_places,adjacency_matrix,node_coords,solutions,bounds,reachable,configs,alpha_shape,collider = load_relevant_info(roadmaps[0],robot_name = 'armbot')
selected_points = sampling_places[reachable,:][solutions>0.5,:]
points_mask = solutions>0.5
# final_trajectory,final_cost = extract_milestone_tour(adjacency_matrix,robot_name,sampling_places,configs,reachable,solutions,node_coords,roadmap = roadmap,full_trajectory = False)
rrt_space = robotplanning.makeSpace(world,robot,edgeCheckResolution=0.1,
                                    ignoreCollisions = [(robot.link(2),robot.link(3)),
                                                    (robot.link(8),robot.link(6))])

In [ ]:
load = True

if not load:
    ik_sols = []
    for point in tqdm(selected_points):
        candidate_ik_sols = find_n_ik_solutions(robot,lamp,collider,world,point,alpha_shape,
                                      restarts = 100,tol = 1e-3,neighborhood = 0.4,float_height = 0.15, n = 1000)
        subsample = iterative_farthest_point(candidate_ik_sols,20)
        ik_sols.append(subsample)
    pickle.dump(ik_sols,open('ik_sols.p','wb'))
else:
    ik_sols = pickle.load(open('ik_sols_200.p','rb'))
set_robot_link_collision_margins(robot,0,collider)
base = robot.link(2)
base.geometry().setCollisionMargin(0)

In [ ]:
# configuration_sequence,ik_sols = euclidean_trelis_solution(selected_points,ik_sols,robot_name)
# configuration_sequence,ik_sols = heuristic_penalized_trelis_solution(selected_points,ik_sols,robot_name,robot,world,penalty = 1.5)
configuration_sequence,ik_sols = heuristic_ee_trelis_solution(selected_points,ik_sols,robot_name,robot,world,lamp,penalty = 10)

In [ ]:
# final_path = []
# for i in tqdm(range(len(configuration_sequence)-1)):
#     curr_cfig = configuration_sequence[i]
#     next_cfig = configuration_sequence[i+1]
#     planner = cspace.MotionPlan(rrt_space,type="rrt*",connectionThreshold=50.0,bidirectional = 1,shortcut = False,knn = 60)  #accepts keyword arguments
#     planner.setEndpoints(curr_cfig,next_cfig)
#     increment = 500               #there is a little overhead for each planMore call, so it is best not to set the increment too low
#     t0 = time.time()
#     tmax = 5
#     no_path = True
#     while time.time() - t0 < tmax or no_path:   #max 20 seconds of planning
#         planner.planMore(increment)
#         path = planner.getPath()
#         if(path is None):
#             if(time.time() - t0 > 10*tmax):
# #                 print('hmmmm planning failed, extending time by 5 seconds')
#                 next_cfig = choice(ik_sols[i+1])
#                 configuration_sequence[i+1] = next_cfig
#                 planner.close()
#                 planner = cspace.MotionPlan(rrt_space,type="rrt*",connectionThreshold=50.0,bidirectional = 1,shortcut = False,knn = 60)  #accepts keyword arguments
#                 planner.setEndpoints(curr_cfig,next_cfig)
#                 print(next_cfig)
#         else:
#             no_path = False
#     if(i != len(configuration_sequence)-1):
# #         print(path)
#         final_path.append(path[:-1])
#     else:
#         final_path.append(path)
#     #     if path is not None:
#     #         print("Solved, path has",len(path),"milestones")
#     #         print("Took time",time.time()-t0)
#     #         break

#     planner.close() 

total_length, appended_path,final_path = calculate_path_length_from_configuration_sequence(world,robot,lamp,configuration_sequence,ik_sols,init_duration = 5)

print('total length = {}'.format(total_length))

In [ ]:
pickle.dump(final_path,open('heuristic_ee_trelis_path_20.p','wb'))

In [ ]:
dt = 0.05
robotTraj = RobotTrajectory(robot,milestones = appended_path).discretize(dt)
linkTraj = robotTraj.getLinkTrajectory(11,dt).getPositionTrajectory([0,0,0])

In [ ]:
vis.add('World',world)
vis.add('trajectory',linkTraj)
vis.show()

In [ ]:
vp = vis.getViewport()
vp.clippingplanes = [0.1,10000]
tform = pickle.load(open('transform.p','rb'))
vp.setTransform(tform)
vp.w = 1853
vp.h = 1123
vis.scene().setViewport(vp)

# Euclidean Trelis Length:
## 86 targets 
### Total_length with RRT* and 20 samples = 67.73 m
### Total length with RRT* and 200 samples = 53.75861546870769 m
## 91 targets
### Total_length with RRT* and 20 samples = 199.67m
### Total length with RRT* and 200 samples = 158.35m

# Heuristic Euclidean Trelis Length:
## 91 targets
## Total_length with RRT* and 20 samples = 180.61m

## 86 targets:
## Total_length with RRT* and 20 samples = 67.39m

# Actual EE distances in the Trelis Length:
## 91 targets
## Total_length with RRT* and 20 samples = 219.804m
## 86 targets
## Total_length with RRT* and 20 samples = 59.89m

# Now for my method:

In [ ]:
roadmaps = glob('./Distance/*/*/*/armbot_roadmap_330_divs.p')

total_dofs  = 11
robot_name = 'armbot'
world,robot,lamp,roadmap,sampling_places,adjacency_matrix,node_coords,solutions,bounds,reachable,configs,alpha_shape,collider = load_relevant_info(roadmaps[0],robot_name = 'armbot')
selected_points = sampling_places[reachable,:][solutions>0,:]
points_mask = solutions>0.5
final_trajectory,final_cost = extract_milestone_tour(adjacency_matrix,robot_name,sampling_places,configs,reachable,solutions,node_coords,roadmap = roadmap,full_trajectory = True)
final_cost
vp = vis.getViewport()
# vp.h = 640
# vp.w = 640
vp.clippingplanes = [0.1,10000]
tform = pickle.load(open('transform.p','rb'))
vp.setTransform(tform)
vp.w = 1853
vp.h = 1123
vis.scene().setViewport(vp)

In [ ]:
final_cost

# Randomly Sampled PRM length:
## total_length = 116.18m
# Randomly Sampled PRM + point-to-point planning
## total_length = 92.38 m

In [ ]:
dt = 0.05
appended_path = []
for i in final_trajectory:
    appended_path.extend(i)
robotTraj = RobotTrajectory(robot,milestones = appended_path).discretize(dt)
linkTraj = robotTraj.getLinkTrajectory(11,dt).getPositionTrajectory([0,0,0])
linkTraj.length()

vis.add('World',world)
vis.add('trajectory',linkTraj)
vis.show()

# Randomly sampled PRM length using the sequential method:

In [ ]:
penalty = 2

set_robot_link_collision_margins(robot,0,collider)
base = robot.link(2)
base.geometry().setCollisionMargin(0)

coarse_space = robotplanning.makeSpace(world,robot,edgeCheckResolution=0.5,
                                    ignoreCollisions = [(robot.link(2),robot.link(3)),
                                                        (robot.link(8),robot.link(6))])
configuration_sequence = []
for i in final_trajectory:
    configuration_sequence.append(i[0])
configuration_sequence.append(final_trajectory[-1][-1])
final_path = []
total_length, appended_path = calculate_path_length_from_configuration_sequence(world,robot,lamp,configuration_sequence,init_duration = 5)


In [ ]:
dt = 0.05
appended_path = []
for i in final_trajectory:
    appended_path.extend(i)
robotTraj = RobotTrajectory(robot,milestones = appended_path).discretize(dt)
linkTraj = robotTraj.getLinkTrajectory(11,dt).getPositionTrajectory([0,0,0])
print(linkTraj.length())

vis.add('World',world)
vis.add('trajectory',linkTraj)
vis.show()

# Heuristic Trelis - Adding cost multiplier on collisions

In [ ]:
def heuristic_penalized_trelis_solution(selected_points,ik_sols,robot_name,penalty = 5):

    coarse_space = robotplanning.makeSpace(world,robot,edgeCheckResolution=0.5,
                                        ignoreCollisions = [(robot.link(2),robot.link(3)),
                                                        (robot.link(8),robot.link(6))])
    distances = np.zeros((selected_points.shape[0]+1,selected_points.shape[0]+1))
    for i in range(selected_points.shape[0]):
        point = selected_points[i]
        distances[1:,i+1] = 1000*np.linalg.norm(selected_points-point,axis = 1)
    euc_distances = distances
    tour = runTSP(euc_distances, '/{}_currTSP'.format(robot_name)) 
    tour = (np.array(tour[1:])-1).tolist()
    trelis_ik = np.array(ik_sols)[tour]
    best_indices = np.zeros((trelis_ik.shape[0]-1,trelis_ik.shape[1]))
    best_cost = np.zeros((trelis_ik.shape[0]-1,trelis_ik.shape[1]))
    for i in tqdm(range(trelis_ik.shape[0]-1)):
        current_iks = trelis_ik[i]
        next_iks = trelis_ik[i+1]
        approx_dist = np.zeros((trelis_ik.shape[1],trelis_ik.shape[1]))
        for j,point in enumerate(next_iks):
            for k,cur_point in enumerate(current_iks):
    #             interpolated_cfigs = interp(cur_point,end,robot)
                if(coarse_space.isVisible(cur_point,point)):
                    approx_dist[j,k] = np.linalg.norm(current_iks-point)
                else:
                    approx_dist[j,k] = penalty*np.linalg.norm(current_iks-point)
    #             approx_dist[j,:] = np.linalg.norm(current_iks-point,axis = 1)

        best_idx = np.argmin(approx_dist,axis = 1)
        cost = np.min(approx_dist, axis = 1)
        if(i != 0):
    #         print(best_cost[i-1,best_idx])
            cost += best_cost[i-1,best_idx]
        best_indices[i,:] = best_idx
        best_cost[i,:] = cost
    j = best_indices.shape[0]-1
    best_idx_path = []
    best_idx_path.append(np.argmin(best_cost[j]).astype(int))
    while(j>=0):
        best_idx_path.append(best_indices[j,best_idx_path[-1]].astype(int))
        j -= 1
    best_idx_path.reverse()
    configuration_sequence = []
    for i,idx in enumerate(best_idx_path):
        configuration_sequence.append(trelis_ik[i,idx])
return configuration_sequence,trelis_ik

# Sanity Check:

In [ ]:
import random

total_dofs  = 11
robot_name = 'armbot'
mesh_file = './data/processed_empty_floor_meters.ply'

bounds,alpha_shape = get_bounds(mesh_file,degree = 0)
# print(bounds)
# bounds = []
# bounds.append(np.min(projection[:,0]))
# bounds.append(np.max(projection[:,0]))
# bounds.append(np.min(projection[:,1]))
# bounds.append(np.max(projection[:,1]))

world,robot,lamp,collider = setup_robot_and_light(robotfile = './data/armbot.rob', mesh_file = mesh_file,float_height = 0.08,bounds = bounds)


In [ ]:
points = np.arange(-6,-0.5,0.4)
selected_points = []
for point in points:
    selected_points.append([3.5,point,0.5])
selected_points = np.array(selected_points)

load = False

if not load:
    ik_sols = []
    for point in tqdm(selected_points):
        candidate_ik_sols = find_n_ik_solutions(robot,lamp,collider,world,point,alpha_shape,
                                      restarts = 100,tol = 1e-3,neighborhood = 0.4,float_height = 0.08, n = 2000)
        subsample = iterative_farthest_point(candidate_ik_sols,200)
        ik_sols.append(subsample)
#     sols_col = ik_sols[0]
#     selected_sol = random.choice(ik_sols[0])
#     for i in range(1,len(ik_sols)):
#         selected_sol[1]+=0.4
#         l = random.choice(list(range(len(ik_sols[i]))))
#         ik_sols[i][l] = selected_sol
    pickle.dump(ik_sols,open('sanity_check_ik_sols_200.p','wb'))
else:
    ik_sols = pickle.load(open('sanity_check_ik_sols_200.p','rb'))
    
set_robot_link_collision_margins(robot,0,collider)
base = robot.link(2)
base.geometry().setCollisionMargin(0)

In [ ]:
# configuration_sequence,ik_sols = euclidean_trelis_solution(selected_points,ik_sols,robot_name)
# configuration_sequence,ik_sols = heuristic_penalized_trelis_solution(selected_points,ik_sols,robot_name,robot,world,penalty = 1.5)
configuration_sequence,ik_sols = heuristic_ee_trelis_solution(selected_points,ik_sols,robot_name,robot,world,lamp,penalty = 10)
rrt_space = robotplanning.makeSpace(world,robot,edgeCheckResolution=0.1,
                                    ignoreCollisions = [(robot.link(2),robot.link(3)),
                                                    (robot.link(8),robot.link(6))])
final_path = []
for i in tqdm(range(len(configuration_sequence)-1)):
    curr_cfig = configuration_sequence[i]
    next_cfig = configuration_sequence[i+1]
    planner = cspace.MotionPlan(rrt_space,type="rrt*",connectionThreshold=50.0,bidirectional = 1,shortcut = True,knn = 60)  #accepts keyword arguments
    planner.setEndpoints(curr_cfig,next_cfig)
    increment = 500               #there is a little overhead for each planMore call, so it is best not to set the increment too low
    t0 = time.time()
    tmax = 2
    no_path = True
    while time.time() - t0 < tmax or no_path:   #max 20 seconds of planning
        planner.planMore(increment)
        path = planner.getPath()
        if(path is None):
            if(time.time() - t0 > 10*tmax):
#                 print('hmmmm planning failed, extending time by 5 seconds')
                next_cfig = choice(ik_sols[i+1])
                configuration_sequence[i+1] = next_cfig
                planner.close()
                planner = cspace.MotionPlan(rrt_space,type="rrt*",connectionThreshold=50.0,bidirectional = 1,shortcut = False,knn = 60)  #accepts keyword arguments
                planner.setEndpoints(curr_cfig,next_cfig)
                print(next_cfig)
        else:
            no_path = False
    if(i != len(configuration_sequence)-1):
#         print(path)
        final_path.append(path[:-1])
    else:
        final_path.append(path)
    #     if path is not None:
    #         print("Solved, path has",len(path),"milestones")
    #         print("Took time",time.time()-t0)
    #         break

    planner.close() 



appended_path = []
for i in final_path:
    appended_path.extend(i)
total_distance = 0 
planner = cspace.MotionPlan(rrt_space,type="rrt*",connectionThreshold=50.0,bidirectional = 1,shortcut = False,knn = 30)  #accepts keyword arguments

for i in range(len(appended_path)-1):
    origin = appended_path[i]
    end = appended_path[i+1]
    total_distance += compute_actual_distance(origin,end,lamp,robot)
total_distance

In [ ]:
appended_path = []
for i in final_path:
    appended_path.extend(i)
total_distance = 0 
planner = cspace.MotionPlan(rrt_space,type="rrt*",connectionThreshold=50.0,bidirectional = 1,shortcut = False,knn = 30)  #accepts keyword arguments

for i in range(len(appended_path)-1):
    origin = appended_path[i]
    end = appended_path[i+1]
    total_distance += compute_actual_distance(origin,end,lamp,robot)
total_distance

pickle.dump(final_path,open('heuristic_ee_trelis_path_20.p','wb'))


dt = 0.05
robotTraj = RobotTrajectory(robot,milestones = appended_path).discretize(dt)
linkTraj = robotTraj.getLinkTrajectory(11,dt).getPositionTrajectory([0,0,0])

vis.add('World',world)
vis.add('trajectory',linkTraj)
vis.show()

vp = vis.getViewport()
# vp.h = 640
# vp.w = 640
vp.clippingplanes = [0.1,10000]
tform = pickle.load(open('transform.p','rb'))
vp.setTransform(tform)
vp.w = 1853
vp.h = 1123
vis.scene().setViewport(vp)

# Multi-PRM -> TSP (mean PRM dist) -> Trellis 

In [ ]:
from planning.robot_cspaces import Robot3DCSpace,UnreachablePointsError,CSpaceObstacleSolver

roadmaps = glob('./Distance/*/*/*/armbot_roadmap_330_divs.p')

total_dofs  = 11
robot_name = 'armbot'
world,robot,lamp,roadmap,sampling_places,adjacency_matrix,node_coords,solutions,bounds,reachable,configs,alpha_shape,collider = load_relevant_info(roadmaps[0],robot_name = 'armbot')
selected_points = sampling_places[reachable,:][solutions>0,:]
points_mask = solutions>0.5
# final_trajectory,final_cost = extract_milestone_tour(adjacency_matrix,robot_name,sampling_places,configs,reachable,solutions,node_coords,roadmap = roadmap,full_trajectory = False)
rrt_space = robotplanning.makeSpace(world,robot,edgeCheckResolution=0.1,
                                    ignoreCollisions = [(robot.link(2),robot.link(3)),
                                                    (robot.link(8),robot.link(6))])
load = True

if not load:
    ik_sols = []
    for point in tqdm(selected_points):
        candidate_ik_sols = find_n_ik_solutions(robot,lamp,collider,world,point,alpha_shape,
                                      restarts = 100,tol = 1e-3,neighborhood = 0.4,float_height = 0.15, n = 2000)
        subsample = iterative_farthest_point(candidate_ik_sols,20)
        ik_sols.append(subsample)
    pickle.dump(ik_sols,open('ik_sols.p','wb'))
else:
    ik_sols = pickle.load(open('ik_sols.p','rb'))
set_robot_link_collision_margins(robot,0,collider)
base = robot.link(2)
base.geometry().setCollisionMargin(0)
sols = 20

In [ ]:
milestones = []
for ik_sol in ik_sols:
    this_conf = ik_sol[0]
    robot.setConfig(this_conf)
    pos = np.array(lamp.getWorldPosition([0,0,0]))
    new_ik_sol = np.zeros((ik_sol.shape[0],ik_sol.shape[1]+3))
    new_ik_sol[:,:ik_sol.shape[1]] = ik_sol
    new_ik_sol[:,ik_sol.shape[1]:] = pos
    milestones.append(new_ik_sol)
milestones = np.array(milestones)
milestones = milestones.reshape(-1,ik_sols[0].shape[-1]+3)
space = Robot3DCSpace(bounds,robot,collider,lamp,milestones,
    base_height_link = 2,
    robot_height = 1.5,
    float_height = 0.15,
    linear_dofs = [0,1],
    angular_dofs = [4,5,6,7,8,9],
    light_local_position  = [0,0,0])
# program = CSpaceObstacleSolver(space,milestones = milestones, initial_points= 5000,steps = 5,max_iters = 10000)
# adjacency_matrix,roadmap,node_coords = program.get_adjacency_matrix_from_milestones()

In [ ]:
# rm = program.planner.getRoadmap()
# adjacency_matrix = program.compute_real_pairwise_distances(rm)

# G_list = rm
# G = nx.Graph()
# G.add_nodes_from(range(len(G_list[0])))
# G.add_edges_from(G_list[1])
# edges = np.array(G_list[1])
# nodes = np.array(G_list[0])
# origins = nodes[edges[:,0],:robot.numLinks()]
# ends = nodes[edges[:,1],:robot.numLinks()]
# weights = program.compute_actual_distance(origins,ends)
# for weight,edge in zip(weights,edges):
#     G.edges[edge]['weight'] = weight
# distances_array = []
# for i in tqdm(range(milestones.shape[0])):
#     distances_dict = dict(nx.algorithms.shortest_path_length(G,source = i, weight = 'weight'))
#     this_distance = []
#     for j in range(milestones.shape[0]):
#         if(j not in distances_dict):
#             this_distance.append(np.inf)
#         else:
#             this_distance.append(distances_dict[j])
#     distances_array.append(this_distance)
# distances = np.array(distances_array)
# G = G
# # return distances

In [ ]:
# pickle.dump(distances,open('200x200x91_distances.p','wb'))
full_distances =  pickle.load(open('20x20x86_distances.p','rb'))

In [ ]:
task_points = int(milestones.shape[0]/sols)
adjacency_matrix = []
for i in range(int(milestones.shape[0]/sols)):
    a = full_distances[sols*i:sols*(i+1)]
    b = np.nan_to_num(a.reshape((sols,task_points,sols)),posinf =10000)
    min_dist = b.max(axis = 2).max(axis = 0)
#     min_dist = np.median(np.median(b,axis = 2),axis = 0)
    adjacency_matrix.append(min_dist)
adjacency_matrix = np.array(adjacency_matrix)

In [ ]:
penalty = 2
coarse_space = robotplanning.makeSpace(world,robot,edgeCheckResolution=0.5,
                                    ignoreCollisions = [(robot.link(2),robot.link(3)),
                                                        (robot.link(8),robot.link(6))])
distances = np.zeros((adjacency_matrix.shape[0]+1,adjacency_matrix.shape[0]+1))
distances[1:,1:] = 1000*adjacency_matrix
euc_distances = distances
tour = runTSP(euc_distances, '/{}_currTSP'.format(robot_name)) 
tour = (np.array(tour[1:])-1).tolist()
trelis_ik = np.array(ik_sols)[tour]
best_indices = np.zeros((trelis_ik.shape[0]-1,trelis_ik.shape[1]))
best_cost = np.zeros((trelis_ik.shape[0]-1,trelis_ik.shape[1]))
for i in tqdm(range(trelis_ik.shape[0]-1)):
    current_iks = trelis_ik[i]
    next_iks = trelis_ik[i+1]
    actual_i = tour.index(i)
    actual_next_i = tour.index(i+1)
    approx_dist = full_distances[sols*(actual_i):sols*(actual_i+1),sols*(actual_next_i):sols*(actual_next_i+1)]
#     break
#     for j,point in enumerate(next_iks):
#         for k,cur_point in enumerate(current_iks):
# #             interpolated_cfigs = interp(cur_point,end,robot)
#             if(coarse_space.isVisible(cur_point,point)):
#                 approx_dist[j,k] = compute_actual_distance(cur_point,point,lamp,robot)
#             else:
#                 approx_dist[j,k] = penalty*compute_actual_distance(cur_point,point,lamp,robot)
#             approx_dist[j,:] = np.linalg.norm(current_iks-point,axis = 1)

    best_idx = np.argmin(approx_dist,axis = 1)
    cost = np.min(approx_dist, axis = 1)
    if(i != 0):
#         print(best_cost[i-1,best_idx])
        cost += best_cost[i-1,best_idx]
    best_indices[i,:] = best_idx
    best_cost[i,:] = cost
j = best_indices.shape[0]-1
best_idx_path = []
best_idx_path.append(np.argmin(best_cost[j]).astype(int))
while(j>=0):
    best_idx_path.append(best_indices[j,best_idx_path[-1]].astype(int))
    j -= 1
best_idx_path.reverse()
configuration_sequence = []
for i,idx in enumerate(best_idx_path):
    configuration_sequence.append(trelis_ik[i,idx])

total_length, appended_path,final_path = calculate_path_length_from_configuration_sequence(world,robot,lamp,configuration_sequence,init_duration = 15)
print(total_length)

In [ ]:
dt = 0.05

robotTraj = RobotTrajectory(robot,milestones = appended_path).discretize(dt)
linkTraj = robotTraj.getLinkTrajectory(11,dt).getPositionTrajectory([0,0,0])
print(linkTraj.length())

vis.add('World',world)
vis.add('trajectory',linkTraj)
vis.show()

## 20 Samples:
### Minimum Distance TSP: 152.20974507926817
### Median Distance TSP: 174.19
### Mean Distance TSP: 129.3 m
## 200 Samples
### minimum distance TSP: 147.8
### mean distance TSP: 161.61
### median distance TSP: 150.77

# Multi-PRM - with iterative redundancy resolution

In [2]:
from planning.robot_cspaces import Robot3DCSpace,UnreachablePointsError,CSpaceObstacleSolver

roadmaps = glob('./Distance/*/*/*/armbot_roadmap_330_divs.p')

total_dofs  = 12
robot_name = 'armbot'
world,robot,lamp,roadmap,sampling_places,adjacency_matrix,node_coords,solutions,bounds,reachable,configs,alpha_shape,collider = load_relevant_info(roadmaps[0],robot_name = 'armbot')
selected_points = sampling_places[reachable,:][solutions>0.5,:]
points_mask = solutions>0.5
# final_trajectory,final_cost = extract_milestone_tour(adjacency_matrix,robot_name,sampling_places,configs,reachable,solutions,node_coords,roadmap = roadmap,full_trajectory = False)
rrt_space = robotplanning.makeSpace(world,robot,edgeCheckResolution=0.1,
                                    ignoreCollisions = [(robot.link(2),robot.link(3)),
                                                    (robot.link(8),robot.link(6))])
load = True

if not load:
    ik_sols = []
    for point in tqdm(selected_points):
        candidate_ik_sols = find_n_ik_solutions(robot,lamp,collider,world,point,alpha_shape,
                                      restarts = 100,tol = 1e-3,neighborhood = 0.4,float_height = 0.15, n = 2000)
        subsample = iterative_farthest_point(candidate_ik_sols,200)
        ik_sols.append(subsample)
    pickle.dump(ik_sols,open('ik_sols_200.p','wb'))
else:
    ik_sols = pickle.load(open('ik_sols_200.p','rb'))
set_robot_link_collision_margins(robot,0,collider)
base = robot.link(2)
base.geometry().setCollisionMargin(0)
sols = 200

/home/motion/.local/lib/python3.8/site-packages/trimesh/repair.py:258: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  cycles = np.array(nx.cycle_basis(g))


RobotWorld::LoadRobot: ./data/armbot.rob
RobParser: Reading robot file ./data/armbot.rob...
0



base height link = 2, lamp linknum = 1



RobParser:    Parsing robot file, 12 links read...
LoadAssimp: Loaded model ./data/primitives/scaled_cylinder.off (66 verts, 128 tris)
LoadAssimp: Loaded model ./data/ur5e/base_link.off (17453 verts, 35004 tris)
LoadAssimp: Loaded model ./data/ur5e/shoulder_link.off (341 verts, 674 tris)
LoadAssimp: Loaded model ./data/ur5e/upper_arm_link.off (598 verts, 1176 tris)
LoadAssimp: Loaded model ./data/ur5e/forearm_link.off (39965 verts, 79927 tris)
LoadAssimp: Loaded model ./data/ur5e/wrist_1_link.off (54492 verts, 108994 tris)
LoadAssimp: Loaded model ./data/ur5e/wrist_2_link.off (60260 verts, 120518 tris)
LoadAssimp: Loaded model ./data/ur5e/wrist_3_link.off (12305 verts, 24586 tris)
LoadAssimp: Loaded model ./data/ur5e/ee_link.off (8 verts, 12 tris)
LoadAssimp: Loaded model ./data/sphere.off (482 verts, 960 tris)
RobParser: Loaded geometries in time 0

In [3]:
milestones = []
for ik_sol in ik_sols:
    this_conf = ik_sol[0]
    robot.setConfig(this_conf)
    pos = np.array(lamp.getWorldPosition([0,0,0]))
    new_ik_sol = np.zeros((ik_sol.shape[0],ik_sol.shape[1]+3))
    new_ik_sol[:,:ik_sol.shape[1]] = ik_sol
    new_ik_sol[:,ik_sol.shape[1]:] = pos
    milestones.append(new_ik_sol)
milestones = np.array(milestones)
milestones = milestones.reshape(-1,ik_sols[0].shape[-1]+3)
space = Robot3DCSpace(bounds,robot,collider,lamp,milestones,
    base_height_link = 2,
    robot_height = 1.5,
    float_height = 0.15,
    linear_dofs = [0,1],
    angular_dofs = [4,5,6,7,8,9],
    light_local_position  = [0,0,0])
# program = CSpaceObstacleSolver(space,milestones = milestones, initial_points= 5000,steps = 5,max_iters = 10000)
# adjacency_matrix,roadmap,node_coords = program.get_adjacency_matrix_from_milestones()

In [4]:
# pickle.dump(distances,open('200x200x91_distances.p','wb'))
full_distances =  pickle.load(open('200x200x86_distances.p','rb'))

In [8]:
import random
task_points = int(milestones.shape[0]/sols)
# adjacency_matrix = []
# for i in range(int(milestones.shape[0]/sols)):
#     a = full_distances[sols*i:sols*(i+1)]
#     b = np.nan_to_num(a.reshape((sols,task_points,sols)),posinf =10000)
#     min_dist = b.mean(axis = 2).mean(axis = 0)
# #     min_dist = np.median(np.median(b,axis = 2),axis = 0)
#     adjacency_matrix.append(min_dist)
# adjacency_matrix = np.array(adjacency_matrix)
selected_iks = np.random.randint(0,sols,task_points)
l = list(range(task_points))
any_change = False
for i in range(100):
    random.shuffle(l)
    any_change = False
    for manifold in l:
#         manifold = random.randint(0,task_points-1)
        selected_indices = selected_iks + (np.array(range(task_points)))*sols
        a = full_distances[sols*manifold:sols*(manifold+1),selected_indices]
        a[:,manifold] = 0
        q = np.quantile(a,q = 0.3,axis = 1)
#         print('before,',(a == 0).sum())
        a[a>q.reshape(-1,1)] = 0
#         print('after,',(a == 0).sum())

        new_best = np.argmin(a.sum(axis = 1))
        if(new_best != selected_iks[manifold]):
            any_change = True
#         print('new best sum - prev_best_sum = {}, new_best, old_best = ({},{})'.format(a[new_best,:].sum()-a[selected_iks[manifold],:].sum(),new_best,selected_iks[manifold]))
            selected_iks[manifold] = new_best
    if(any_change == False):
        print('rounds until convergence = {}'.format(i))
        break
#     break

rounds until convergence = 6


In [9]:
adjacency_matrix = full_distances[selected_indices,:][:,selected_indices]
adjacency_matrix.shape

(86, 86)

In [ ]:
penalty = 2
coarse_space = robotplanning.makeSpace(world,robot,edgeCheckResolution=0.5,
                                    ignoreCollisions = [(robot.link(2),robot.link(3)),
                                                        (robot.link(8),robot.link(6))])
distances = np.zeros((adjacency_matrix.shape[0]+1,adjacency_matrix.shape[0]+1))
distances[1:,1:] = 1000*adjacency_matrix
euc_distances = distances
tour = runTSP(euc_distances, '/{}_currTSP'.format(robot_name)) 
tour = (np.array(tour[1:])-1).tolist()
trelis_ik = np.array(ik_sols)[tour]
best_indices = np.zeros((trelis_ik.shape[0]-1,trelis_ik.shape[1]))
best_cost = np.zeros((trelis_ik.shape[0]-1,trelis_ik.shape[1]))
for i in tqdm(range(trelis_ik.shape[0]-1)):
    current_iks = trelis_ik[i]
    next_iks = trelis_ik[i+1]
    actual_i = tour.index(i)
    actual_next_i = tour.index(i+1)
    approx_dist = np.zeros((sols,sols))
#     approx_dist = full_distances[sols*(actual_i):sols*(actual_i+1),sols*(actual_next_i):sols*(actual_next_i+1)]
#     break
#     for j,point in enumerate(next_iks):
#         for k,cur_point in enumerate(current_iks):
# #             interpolated_cfigs = interp(cur_point,end,robot)
#             if(coarse_space.isVisible(cur_point,point)):
#                 approx_dist[j,k] = compute_actual_distance(cur_point,point,lamp,robot)
#             else:
#                 approx_dist[j,k] = penalty*compute_actual_distance(cur_point,point,lamp,robot)
#             approx_dist[j,:] = np.linalg.norm(current_iks-point,axis = 1)
    for j,point in enumerate(next_iks):
#         for k,cur_point in enumerate(current_iks):
#             interpolated_cfigs = interp(cur_point,end,robot)
#             if(coarse_space.isVisible(cur_point,point)):
#                 approx_dist[j,k] = np.linalg.norm(cur_point-point)
#             else:
#                 approx_dist[j,k] = penalty*compute_actual_distance(cur_point,point,lamp,robot)
        approx_dist[j,:] = np.linalg.norm(current_iks-point,axis = 1)

    best_idx = np.argmin(approx_dist,axis = 1)
    cost = np.min(approx_dist, axis = 1)
    if(i != 0):
#         print(best_cost[i-1,best_idx])
        cost += best_cost[i-1,best_idx]
    best_indices[i,:] = best_idx
    best_cost[i,:] = cost
j = best_indices.shape[0]-1
best_idx_path = []
best_idx_path.append(np.argmin(best_cost[j]).astype(int))
while(j>=0):
    best_idx_path.append(best_indices[j,best_idx_path[-1]].astype(int))
    j -= 1
best_idx_path.reverse()
configuration_sequence = []
for i,idx in enumerate(best_idx_path):
    configuration_sequence.append(trelis_ik[i,idx])

total_distance,appended_path,final_path = calculate_path_length_from_configuration_sequence(world,robot,lamp,configuration_sequence,ik_sols,init_duration = 30)
# rrt_space = robotplanning.makeSpace(world,robot,edgeCheckResolution=0.1,
#                                     ignoreCollisions = [(robot.link(2),robot.link(3)),
#                                                     (robot.link(8),robot.link(6))])
# final_path = []
# for i in tqdm(range(len(configuration_sequence)-1)):
#     curr_cfig = configuration_sequence[i]
#     next_cfig = configuration_sequence[i+1]
#     if(rrt_space.isVisible(curr_cfig,next_cfig)):
#         path = [curr_cfig,next_cfig]
#     else:
#         planner = cspace.MotionPlan(rrt_space,type="sbl",connectionThreshold=50.0,bidirectional = 1,shortcut = True,restart = True, knn = 60)  #accepts keyword arguments
#         planner.setEndpoints(curr_cfig,next_cfig)
#         increment = 500               #there is a little overhead for each planMore call, so it is best not to set the increment too low
#         t0 = time.time()
#         tmax = 30
#         no_path = True
#         while time.time() - t0 < tmax or no_path:   #max 20 seconds of planning
#             planner.planMore(increment)
#             path = planner.getPath()
#             if(path is None):
#                 if(time.time() - t0 > 10*tmax):
#     #                 print('hmmmm planning failed, extending time by 5 seconds')
#                     next_cfig = choice(ik_sols[i+1])
#                     configuration_sequence[i+1] = next_cfig
#                     planner.close()
#                     planner = cspace.MotionPlan(rrt_space,type="sbl",connectionThreshold=50.0,bidirectional = 1,shortcut = True,restart = True,knn = 60)  #accepts keyword arguments
#                     planner.setEndpoints(curr_cfig,next_cfig)
#                     print(next_cfig)
#             else:
#                 no_path = False
#     if(i != len(configuration_sequence)-1):
# #         print(path)
#         final_path.append(path[:-1])
#     else:
#         final_path.append(path)
#     #     if path is not None:
#     #         print("Solved, path has",len(path),"milestones")
#     #         print("Took time",time.time()-t0)
#     #         break

# #     planner.close() 



# appended_path = []
# for i in final_path:
#     appended_path.extend(i)
# total_distance = 0 
# planner = cspace.MotionPlan(rrt_space,type="sbl",connectionThreshold=50.0,bidirectional = 1,shortcut = True,restart = True,knn = 30)  #accepts keyword arguments

# for i in range(len(appended_path)-1):
#     origin = appended_path[i]
#     end = appended_path[i+1]
#     total_distance += compute_actual_distance(origin,end,lamp,robot)
total_distance






 /armbot_currTSP    



PARAMETER_FILE = /home/motion/Neural-Multigoal-Path-Planning/armbot_currTSP.par
PROBLEM_FILE = /home/motion/Neural-Multigoal-Path-Planning/armbot_currTSP.tsp



 44%|██████████████████████████▉                                   | 37/85 [00:00<00:00, 368.20it/s]

Successes/Runs = 0/4 
Cost.min = 77403, Cost.avg = 77403.00, Cost.max = 77403
Gap.min = 0.0000%, Gap.avg = 0.0000%, Gap.max = 0.0000%
Trials.min = 87, Trials.avg = 87.0, Trials.max = 87
Time.min = 0.05 sec., Time.avg = 0.06 sec., Time.max = 0.06 sec.




  0%|                                                                        | 0/85 [00:00<?, ?it/s]
Exception in thread Thread-9:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/motion/.local/lib/python3.8/site-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.8/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration


swig/python detected a memory leak of type 'std::string *', no destructor found.
swig/python detected a memory leak of type 'std::string *', no destructor found.
swig/python detected a memory leak of type 'std::string *', no destructor found.
swig/python detected a memory leak of type 'std::string *', no destructor found.
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm plannin

...

hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extendin

swig/python dehmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning fa

hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extendin

swig/python detected a memory leak of type 'std::string *', no destructhmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm plann

hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extending time by 5 seconds
hmmmm planning failed, extendin

In [ ]:
total_distance

# WTF is going on

In [ ]:
#major_imports
from glob import glob 
from klampt import IKObjective
from klampt import WorldModel,Geometry3D
from klampt import vis
from klampt.math import so3,se3
from klampt.io import resource
from klampt.math import vectorops,so3
from klampt.model.collide import WorldCollider
from klampt.model import ik
from klampt.plan.cspace import CSpace,MotionPlan
from klampt.model.trajectory import RobotTrajectory,Trajectory
from klampt.math.vectorops import interpolate
from klampt.plan.cspace import CSpace,MotionPlan
from planning.disinfection3d import DisinfectionProblem
from planning.robot_cspaces import Robot3DCSpace,CSpaceObstacleSolver,UnreachablePointsError
import pickle
import os
import numpy as np
from planning.tsp_solver_wrapper import runTSP
import networkx as nx
import klampt
from klampt.plan.robotcspace import RobotCSpace
from klampt.plan import cspace
from klampt.model import collide
import klampt
from klampt.plan import cspace,robotplanning
from klampt.io import resource
import time
import trimesh as tm
from alphashape import alphashape
import shapely
from planning.auxiliary_functions import setup_robot_and_light,get_bounds,load_relevant_info
from planning.auxiliary_functions import extract_milestone_tour,collisionChecker,find_n_ik_solutions
from planning.auxiliary_functions import iterative_farthest_point,euclidean_trelis_solution,set_robot_link_collision_margins
from planning.auxiliary_functions import interp,compute_actual_distance,heuristic_penalized_trelis_solution
from planning.auxiliary_functions import heuristic_ee_trelis_solution
from tqdm import tqdm
from random import choice
from klampt.model.trajectory import RobotTrajectory,Trajectory
from planning.robot_cspaces import Robot3DCSpace,UnreachablePointsError,CSpaceObstacleSolver

roadmaps = glob('./Distance/*/*/*/armbot_roadmap_330_divs.p')

robot_name = 'armbot'
tmin = 0.5
world,robot,lamp,roadmap,sampling_places,adjacency_matrix,node_coords,solutions,bounds,reachable,configs,alpha_shape,collider = load_relevant_info(roadmaps[0],robot_name = 'armbot')
selected_points = sampling_places[reachable,:][solutions>tmin,:]
total_dofs  = robot.numLinks()

points_mask = solutions>tmin
# final_trajectory,final_cost = extract_milestone_tour(adjacency_matrix,robot_name,sampling_places,configs,reachable,solutions,node_coords,roadmap = roadmap,full_trajectory = False)
rrt_space = robotplanning.makeSpace(world,robot,edgeCheckResolution=0.1,
                                    ignoreCollisions = [(robot.link(2),robot.link(3)),
                                                    (robot.link(8),robot.link(6))])
load = True

if not load:
    ik_sols = []
    for point in tqdm(selected_points):
        candidate_ik_sols = find_n_ik_solutions(robot,lamp,collider,world,point,alpha_shape,
                                      restarts = 100,tol = 1e-3,neighborhood = 0.4,float_height = 0.15, n = 2000)
        subsample = iterative_farthest_point(candidate_ik_sols,200)
        ik_sols.append(subsample)
    pickle.dump(ik_sols,open('ik_sols_200.p','wb'))
else:
    ik_sols = pickle.load(open('ik_sols.p','rb'))
set_robot_link_collision_margins(robot,0,collider)
base = robot.link(2)
base.geometry().setCollisionMargin(0)
sols = 200

milestones = []
for ik_sol in ik_sols:
    this_conf = ik_sol[0]
    robot.setConfig(this_conf)
    pos = np.array(lamp.getWorldPosition([0,0,0]))
    new_ik_sol = np.zeros((ik_sol.shape[0],ik_sol.shape[1]+3))
    new_ik_sol[:,:ik_sol.shape[1]] = ik_sol
    new_ik_sol[:,ik_sol.shape[1]:] = pos
    milestones.append(new_ik_sol)
milestones = np.array(milestones)
milestones = milestones.reshape(-1,ik_sols[0].shape[-1]+3)
space = Robot3DCSpace(bounds,robot,collider,lamp,milestones,
    base_height_link = 2,
    robot_height = 1.5,
    float_height = 0.15,
    linear_dofs = [0,1],
    angular_dofs = [4,5,6,7,8,9],
    light_local_position  = [0,0,0])
program = CSpaceObstacleSolver(space,milestones = milestones, initial_points= 2,steps = 10,max_iters = 10)
adjacency_matrix,roadmap,node_coords = program.get_adjacency_matrix_from_milestones()

# full_distances = adjacency_matrix

# pickle.dump(full_distances,open('20x20x86_distances.p','wb'))

In [ ]:
rm = program.planner.getRoadmap()
# self.adjacency_matrix = self.compute_real_pairwise_distances(rm)

In [ ]:
rm[1]

In [ ]:
b = program.milestones[32]

In [ ]:
interp = program.space.interp(a,b)

In [ ]:
interp